In [461]:
import numpy as np
import pandas as pd
import pytse_client as tse
import jdatetime
from datetime import datetime

from scipy.ndimage.interpolation import shift

/tmp/ipykernel_28779/3565205630.py:7: DeprecationWarning:

Please use `shift` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.



# get data from the web

In [462]:
#tickers = tse.download(symbols="all", write_to_csv=True,  include_jdate=True)
Price1 = pd.read_csv("/home/khaled/Project/AdjustedPrice_P/tickers_data/قشکر.csv")
Price = Price1[Price1['volume']>100].copy()

## We want to adjust the price from the 1400-01-01 till the 1401-12-01 with respect to all capital injections that occurred in this interval

In [463]:
FirstDayOfInterval = '1398-01-01'
LastDayOfInterval = '1401-12-01'

gregorian_date = jdatetime.date(int(LastDayOfInterval[0:4]),
                                int(LastDayOfInterval[5:7]),int(LastDayOfInterval[8:10])).togregorian()
Fgregorian_date = jdatetime.date(int(FirstDayOfInterval[0:4]),
                                 int(FirstDayOfInterval[5:7]),int(FirstDayOfInterval[8:10])).togregorian()

def DateT(a):
    return(datetime.fromisoformat((a)))
Price['date'] = Price['date'].apply(lambda x: DateT(x))
PriceFilter = Price[(Price['date'] > pd.Timestamp(pd.to_datetime(Fgregorian_date))) & (Price['date'] < pd.Timestamp(pd.to_datetime(gregorian_date)))]


In [464]:
PriceFilter.iloc[-1, Loc]

2242.0

In [465]:
Prod

6.458554280681043

In [466]:
Ratio =  PriceFilter['adjClose']/ PriceFilter['yesterday']
CumProd = Ratio.cumprod()
Prod = Ratio.prod()
AdjusetCumProd = CumProd/Prod
Loc = PriceFilter.columns.get_loc('adjClose')
a = PriceFilter.iloc[-1, Loc]*AdjusetCumProd
PriceFilter.insert(Loc+1,"AdjustedPrice", a, True)
PriceFilter


,date,open,high,low,adjClose,AdjustedPrice,value,volume,count,yesterday,close,jdate
2538,2019-03-25,4120.0,4291.0,4120.0,4250.0,360.981202,5105254415,1201177,338,4087.0,4291.0,1398-01-05
2539,2019-03-26,4300.0,4462.0,4230.0,4439.0,377.034248,6434626730,1449560,402,4250.0,4462.0,1398-01-06
2540,2019-03-27,4595.0,4600.0,4280.0,4357.0,370.069435,7586662015,1741354,444,4439.0,4300.0,1398-01-07
2541,2019-03-30,4400.0,4400.0,4140.0,4235.0,359.707151,7701174418,1818329,505,4357.0,4250.0,1398-01-10
2542,2019-03-31,4262.0,4359.0,4182.0,4253.0,361.236012,3667908326,862520,240,4235.0,4238.0,1398-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...
3421,2023-02-12,2260.0,2300.0,2190.0,2255.0,2255.000000,17083085900,7576780,315,2250.0,2235.0,1401-11-23
3422,2023-02-13,2220.0,2338.0,2220.0,2259.0,2259.000000,14418759035,6383100,349,2255.0,2231.0,1401-11-24
3423,2023-02-14,2226.0,2298.0,2201.0,2239.0,2239.000000,18905339608,8443787,314,2259.0,2240.0,1401-11-25
3424,2023-02-15,2267.0,2294.0,2212.0,2250.0,2250.000000,13114955834,5825618,288,2239.0,2255.0,1401-11-26


In [468]:
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = go.Figure()
fig.add_trace(go.Scatter(x=PriceFilter['date'], y=PriceFilter['adjClose'],name="BeforAdjust"))
fig.add_trace(go.Scatter(x=PriceFilter['date'], y=PriceFilter['AdjustedPrice'], name="AfterAdjust"))
